In [10]:
import gym
from gym.envs.registration import register
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt



In [18]:
# bandits
class contextual_bandit():
    def __init__(self):
        self.state = 0
        #lower number, the better. Therefore optimal [4,2,1]
        self.bandits = np.array([ [0.2, 0 ,-0.0, -5 ], [0.1, -5,1,0.25],[-5,5,5,5] ])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
    
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits))
        return self.state
    
    def pullArm(self,action):
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            return 1
        else:
            return -1

In [19]:
# agent
class agent():
    def __init__(self, lr , s_size, a_size):
        
        self.state_in = tf.placeholder(shape=[1], dtype = tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
        
        output = slim.fully_connected(state_in_OH,a_size,
                                      biases_initializer=None,
                                      activation_fn= tf.nn.sigmoid,
                                      weights_initializer = tf.ones_initializer())
            
        self.output = tf.reshape(output,[-1])
        self.chosen_action = tf.argmax(self.output, 0)
        
        self.reward_holder = tf.placeholder( shape=[1], dtype = tf.float32)
        
        self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
        
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        
        self.update = optimizer.minimize(self.loss)

In [33]:
tf.reset_default_graph()

cBandit = contextual_bandit()

myAgent = agent(lr=0.001, s_size = cBandit.num_bandits, a_size=cBandit.num_actions)

weights = tf.trainable_variables()[0]
total_episodes = 10000


total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions])
e = 0.1
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit()
        
        
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action,feed_dict={myAgent.state_in:[s]})
            
        reward = cBandit.pullArm(action)
        
        #update network
        _,ww = sess.run([myAgent.update, weights]
                              , feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]})
        total_reward[s,action] += reward
        if i % 509 == 0:
            print("Running reward for the " + str(cBandit.num_bandits) + 
                  " bandits: \n " + str(total_reward))
        i+=1
    for a in range(cBandit.num_bandits):    
        print("The agent thinks action " + str(np.argmax(ww[a])+1) + "for bandit " + str(a+1) + " is the most promising....")
        if np.argmax(ww[a]) == np.argmax(-np.array(cBandit.bandits[a])):
            print( "and it was right!")
        else:
            print("Wrong!")
            
    trained_weights = tf.trainable_variables()
    twout = sess.run(trained_weights)          
    for var, val in zip(trained_weights,twout):
        print(var.name, val)

Running reward for the 3 bandits: 
 [[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
Running reward for the 3 bandits: 
 [[ -1.  -2.  -4. 160.]
 [ -2. 151.  -3.   0.]
 [165.  -2.  -2.  -6.]]
Running reward for the 3 bandits: 
 [[ -2.   0.  -1. 325.]
 [ -3. 319.  -6.  -1.]
 [308.  -4.  -7.  -7.]]
Running reward for the 3 bandits: 
 [[ -3.   5.   0. 487.]
 [ -2. 475. -10.  -3.]
 [460.  -5. -10. -12.]]
Running reward for the 3 bandits: 
 [[ -5.   7.   0. 651.]
 [ -1. 629. -14.  -6.]
 [613. -11. -14. -14.]]
Running reward for the 3 bandits: 
 [[ -5.  11.   2. 818.]
 [ -2. 773. -18.  -8.]
 [773. -15. -20. -15.]]
Running reward for the 3 bandits: 
 [[ -5.  13.   3. 978.]
 [ -4. 915. -23. -10.]
 [937. -16. -30. -19.]]
Running reward for the 3 bandits: 
 [[-6.000e+00  1.100e+01  3.000e+00  1.144e+03]
 [-1.000e+00  1.069e+03 -2.500e+01 -1.100e+01]
 [ 1.097e+03 -1.900e+01 -3.600e+01 -2.200e+01]]
Running reward for the 3 bandits: 
 [[  -8.   10.    2. 1296.]
 [   0. 1231.  -25.  -13.]
 [

In [30]:

#tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

RuntimeError: Attempted to use a closed Session.